In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install openai
import openai
import os
from openai import OpenAI
import pandas as pd
import json
!pip install quickchart.io
from quickchart import QuickChart
import re
from IPython.display import Image, display

import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import MinMaxScaler
from scipy.stats import zscore, ttest_ind

import csv
from datetime import datetime
from scipy import stats

Mounted at /content/drive
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.9/335.9 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.2 MB/s eta 0:00:00


In [ ]:
%env OPENAI_API_KEY= xxx # use your open ai api key
openai.api_key = os.getenv("OPENAI_API_KEY")

env: OPENAI_API_KEY=xxx # use your open ai api key


In [ ]:
# Global variable for the model
MODEL = "gpt-4o"
GEN_TEMPERTURE = 0.0
ACC_TEMPERTURE = 0.0

# Load csv

In [ ]:

# Define the path to the CSV files
base_path = '/content/drive/MyDrive/2024 May Eduview/sample/testing_db_id_matched/'

# Initialize an empty dictionary to store the DataFrames
dataframes = {}

# Iterate over all files in the specified directory
for file_name in os.listdir(base_path):
    if file_name.endswith('.csv'):
        # Extract the table name by removing the '.csv' extension
        table_name = os.path.splitext(file_name)[0]
        # Read the CSV file into a DataFrame
        df = pd.read_csv(os.path.join(base_path, file_name))
        # Store the DataFrame in the dictionary with the table name as the key
        dataframes[table_name] = df

# Display the keys of the dictionary to verify the table names
print(dataframes.keys())
dataframes['Students_routes']['Student_absence_days_per_year']

dict_keys(['Routes_1', 'Bus_incident', 'Trip', 'Bus_routes', 'Bus_routes_2', 'Schedules_1', 'Students_buses', 'Bus_data_1', 'Students_1', 'Students_routes', 'Schedules_2'])


,Student_absence_days_per_year
0,5
1,6
2,3
3,2
4,0
5,1
6,9
7,0
8,1
9,10


# predefined function

In [ ]:
def rename_columns(df, new_column_names):
    if new_column_names:
        df.columns = new_column_names
    return df

def sum_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].sum() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def mean_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].mean() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def max_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].max() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def min_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].min() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def std_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].std() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def var_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].var() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def median_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].median() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def product_columns(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].prod() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def sum_of_squares(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": (database_dict[table][column] ** 2).sum() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def mode_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].mode()[0] for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def range_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].max() - database_dict[table][column].min() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def sum_abs_diff_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    if len(columns) != 2:
        raise ValueError("Sum of absolute differences computation requires exactly 2 columns.")
    col1 = database_dict[columns[0][0]][columns[0][1]]
    col2 = database_dict[columns[1][0]][columns[1][1]]
    result = (col1 - col2).abs().sum()
    result_df = pd.Series({'sum_abs_diff': result}, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def cumulative_sum_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].cumsum() for i, (table, column) in enumerate(columns)}
    result_df = pd.DataFrame(results)
    return rename_columns(result_df, output_column_names)

def regression(database_dict, params):
    x_columns = params['x']
    y_column = params['y']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    X = pd.concat([database_dict[table][column].rename(f"col_{i}") for i, (table, column) in enumerate(x_columns)], axis=1)
    y = database_dict[y_column[0]][y_column[1]].rename("y")

    model = LinearRegression()
    model.fit(X, y)
    predictions = model.predict(X)
    mse = mean_squared_error(y, predictions)

    coefficients = pd.Series(model.coef_, index=X.columns, name='Coefficient')
    intercept = pd.Series({'Intercept': model.intercept_}, name='Coefficient')
    mse_series = pd.Series({'MSE': mse}, name='Coefficient')
    results = pd.concat([intercept, coefficients, mse_series])

    result_df = results.to_frame(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def z_score(database_dict, params):
    column = params['column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    data = database_dict[column[0]][column[1]]
    z_scores = zscore(data)
    result_df = pd.DataFrame({f'col_0': z_scores}, index=data.index)
    return rename_columns(result_df, output_column_names)

def t_test(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    if len(columns) != 2:
        raise ValueError("T-test computation requires exactly 2 columns.")
    group1 = database_dict[columns[0][0]][columns[0][1]].dropna()
    group2 = database_dict[columns[1][0]][columns[1][1]].dropna()
    t_stat, p_value = ttest_ind(group1, group2)
    result_df = pd.DataFrame({'T-Statistic': [t_stat], 'P-Value': [p_value]})
    return rename_columns(result_df, output_column_names)

def correlation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    if len(columns) != 2:
        raise ValueError("Correlation computation requires exactly 2 columns.")
    data1 = database_dict[columns[0][0]][columns[0][1]]
    data2 = database_dict[columns[1][0]][columns[1][1]]
    correlation_value = data1.corr(data2)
    result_df = pd.DataFrame({'correlation': [correlation_value]}, index=[0])
    return rename_columns(result_df, output_column_names)

def skewness_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].skew() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def quantile_computation(database_dict, params):
    table, column = params['category_column']
    quantile = params['quantile']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    data = database_dict[table][column]
    result = data.quantile(quantile)
    result_df = pd.Series({f'col_0': result}, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def normalization(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    scaler = MinMaxScaler()
    data = pd.concat([database_dict[table][column] for table, column in columns], axis=1)
    normalized_data = scaler.fit_transform(data)
    normalized_df = pd.DataFrame(normalized_data, columns=[f"col_{i}" for i in range(len(columns))])
    return rename_columns(normalized_df, output_column_names)

def log_transformation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": np.log(database_dict[table][column]) for i, (table, column) in enumerate(columns)}
    result_df = pd.DataFrame(results)
    return rename_columns(result_df, output_column_names)

def exp_transformation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": np.exp(database_dict[table][column]) for i, (table, column) in enumerate(columns)}
    result_df = pd.DataFrame(results)
    return rename_columns(result_df, output_column_names)

def pct_change_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].pct_change() for i, (table, column) in enumerate(columns)}
    result_df = pd.DataFrame(results)
    return rename_columns(result_df, output_column_names)

def covariance_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    if len(columns) != 2:
        raise ValueError("Covariance computation requires exactly 2 columns.")
    data1 = database_dict[columns[0][0]][columns[0][1]]
    data2 = database_dict[columns[1][0]][columns[1][1]]
    result = data1.cov(data2)
    result_df = pd.Series({'covariance': result}, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def iqr_computation(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].quantile(0.75) - database_dict[table][column].quantile(0.25) for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def categories_sum(database_dict, params):
    group_by_column = params['group_by_column']
    target_column = params['target_column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = group_by_column[0]
    group_by_col = group_by_column[1]
    target_col = target_column[1]

    grouped = database_dict[table].groupby(group_by_col)[target_col].sum()
    result_df = grouped.reset_index(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def categories_mean(database_dict, params):
    group_by_column = params['group_by_column']
    target_column = params['target_column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = group_by_column[0]
    group_by_col = group_by_column[1]
    target_col = target_column[1]

    grouped = database_dict[table].groupby(group_by_col)[target_col].mean()
    result_df = grouped.reset_index(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def categories_min(database_dict, params):
    group_by_column = params['group_by_column']
    target_column = params['target_column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = group_by_column[0]
    group_by_col = group_by_column[1]
    target_col = target_column[1]

    grouped = database_dict[table].groupby(group_by_col)[target_col].min()
    result_df = grouped.reset_index(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def categories_max(database_dict, params):
    group_by_column = params['group_by_column']
    target_column = params['target_column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = group_by_column[0]
    group_by_col = group_by_column[1]
    target_col = target_column[1]

    grouped = database_dict[table].groupby(group_by_col)[target_col].max()
    result_df = grouped.reset_index(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def categories_count(database_dict, params):
    group_by_column = params['group_by_column']
    target_column = params['target_column']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = group_by_column[0]
    group_by_col = group_by_column[1]
    target_col = target_column[1]

    grouped = database_dict[table].groupby(group_by_col)[target_col].count()
    result_df = grouped.reset_index(name=output_table_name)
    return rename_columns(result_df, output_column_names)

def numerical_sum(database_dict, params):
    category_column = params['category_column']
    target_column = params['target_column']
    splits = params['splits']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = category_column[0]
    cat_col = category_column[1]
    target_col = target_column[1]

    data = database_dict[table]
    data['bins'] = pd.cut(data[cat_col], bins=splits)
    grouped = data.groupby('bins')[target_col].sum().reset_index()
    grouped.columns = ['range', 'sum']

    result_df = grouped.rename(columns={'sum': output_table_name})
    return rename_columns(result_df, output_column_names)

def numerical_mean(database_dict, params):
    category_column = params['category_column']
    target_column = params['target_column']
    splits = params['splits']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    table = category_column[0]
    cat_col = category_column[1]
    target_col = target_column[1]

    data = database_dict[table]
    data['bins'] = pd.cut(data[cat_col], bins=splits)
    grouped = data.groupby('bins')[target_col].mean().reset_index()
    grouped.columns = ['range', 'mean']

    result_df = grouped.rename(columns={'mean': output_table_name})
    return rename_columns(result_df, output_column_names)

def count(database_dict, params):
    columns = params['columns']
    output_table_name = params['output_table_name']
    output_column_names = params['output_column_names']

    results = {f"col_{i}": database_dict[table][column].count() for i, (table, column) in enumerate(columns)}
    result_df = pd.Series(results, name=output_table_name).to_frame().T
    return rename_columns(result_df, output_column_names)

def group_by_columns(database_dict, params):
    table = params['table']
    group_by_columns = params['group_by_columns']
    df = database_dict[table].copy()
    return df.groupby(group_by_columns, as_index=False).apply(lambda x: x).reset_index(drop=True)

def join_tables(database_dict, params):
    columns = params['columns']
    how = params.get('how', 'inner')
    output_table_name = params['output_table_name']

    if len(columns) < 2:
        raise ValueError("At least two columns are required for a join operation.")

    table1, column1 = columns[0]
    result = database_dict[table1]

    for table, column in columns[1:]:
        result = result.merge(database_dict[table], left_on=column1, right_on=column, how=how)
        column1 = column

    result = result.drop_duplicates()
    result.name = output_table_name

    return result

def arrange(database_dict, params):
    columns = params['columns']
    table_name, column_name = columns[0]
    descending = params.get('descending', False)
    output_table_name = params['output_table_name']

    df = database_dict[table_name]
    sorted_df = df.sort_values(by=column_name, ascending=not descending)
    sorted_df.name = output_table_name

    return sorted_df


def filter(dataframes, params):
    table_name = params['table']
    column = params['column']
    condition = params['condition']
    output_table_name = params['output_table_name']

    if table_name in dataframes:
        df = dataframes[table_name]
        filtered_df = df.query(f"{column}{condition}")
        filtered_df.name = output_table_name
    else:
        print(f"Table {table_name} not found in dataframes.")
        return pd.DataFrame()

    return filtered_df

def select_columns(dataframes, params):
    table_name = params['table']
    columns = params['column']
    output_table_name = params['output_table_name']

    if table_name in dataframes:
        df = dataframes[table_name]
        selected_df = df[columns]
        selected_df.name = output_table_name
    else:
        print(f"Table {table_name} not found in dataframes.")
        return pd.DataFrame()

    return selected_df

def calculate_time_difference(dataframes, params):
    from datetime import datetime

    table1 = params["time1"]["table"]
    column1 = params["time1"]["column"]
    table2 = params["time2"]["table"]
    column2 = params["time2"]["column"]

    times1 = dataframes[table1][column1]
    times2 = dataframes[table2][column2]

    time_differences = []

    if len(times1) == len(times2):
      for t1, t2 in zip(times1, times2):
          # Parsing the times
          time1 = datetime.strptime(t1, '%I:%M %p')
          time2 = datetime.strptime(t2, '%I:%M %p')

          # Calculating the difference in minutes
          time_diff = (time1 - time2).total_seconds() / 60

          # Append the time difference to the list
          time_differences.append(time_diff)

    else:
        raise ValueError("Two columns need to have same number of rows")

    return time_differences
def mutate(dataframes, params):
    table_name = params['table']
    new_columns = params['new_columns']
    output_table_name = params['output_table_name']

    if table_name in dataframes:
        df = dataframes[table_name].copy()
        for new_col_name, col_info in new_columns.items():
            if isinstance(col_info, str) and col_info.startswith("lambda"):
                col_info = eval(col_info)
                df[new_col_name] = col_info(df)
            elif isinstance(col_info, dict):
                for old_col_name, col_func in col_info.items():
                    df[new_col_name] = df[old_col_name].apply(eval(col_func))
                    df[new_col_name] = df[old_col_name].apply(eval(col_func))
            else:
                df[new_col_name] = col_info(df)

        df.name = output_table_name
    else:
        print(f"Table {table_name} not found in dataframes.")
        return pd.DataFrame()

    return df


In [ ]:
computations_supported = {
    'sum': {
        'function': sum_columns,
        'description': 'Compute the sum of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "sum_table", "output_column_names": ["sum_column1", "sum_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'mean': {
        'function': mean_columns,
        'description': 'Compute the mean of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "mean_table", "output_column_names": ["mean_column1", "mean_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'max': {
        'function': max_columns,
        'description': 'Compute the maximum value of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "max_table", "output_column_names": ["max_column1", "max_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'min': {
        'function': min_columns,
        'description': 'Compute the minimum value of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "min_table", "output_column_names": ["min_column1", "min_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'std': {
        'function': std_columns,
        'description': 'Compute the standard deviation of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "std_table", "output_column_names": ["std_column1", "std_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'var': {
        'function': var_columns,
        'description': 'Compute the variance of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "var_table", "output_column_names": ["var_column1", "var_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'median': {
        'function': median_columns,
        'description': 'Compute the median of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "median_table", "output_column_names": ["median_column1", "median_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'product': {
        'function': product_columns,
        'description': 'Compute the product of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "product_table", "output_column_names": ["product_column1", "product_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'sum_of_squares': {
        'function': sum_of_squares,
        'description': 'Compute the sum of squares of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "sum_of_squares_table", "output_column_names": ["sum_of_squares_column1", "sum_of_squares_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'mode': {
        'function': mode_computation,
        'description': 'Compute the mode of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "mode_table", "output_column_names": ["mode_column1", "mode_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'range': {
        'function': range_computation,
        'description': 'Compute the range (max - min) of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "range_table", "output_column_names": ["range_column1", "range_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'sum_abs_diff': {
        'function': sum_abs_diff_computation,
        'description': 'Compute the sum of absolute differences between two columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"]], "output_table_name": "sum_abs_diff_table", "output_column_names": ["sum_abs_diff_column1"]}. The number of output columns should be the same as the number of input column pairs.'
    },
    'cumulative_sum': {
        'function': cumulative_sum_computation,
        'description': 'Compute the cumulative sum of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "cumulative_sum_table", "output_column_names": ["cumulative_sum_column1", "cumulative_sum_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'regression': {
        'function': regression,
        'description': 'Perform linear regression on the specified columns across tables in the database. JSON format: {"x": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "y": ["table_name_y", "column_name_y"], "output_table_name": "regression_table", "output_column_names": ["regression_coefficient", "regression_intercept"]}. The number of output columns should be two: one for the regression coefficient and one for the intercept.'
    },
    'z_score': {
        'function': z_score,
        'description': 'Compute the z-score of the specified column across tables in the database. JSON format: {"column": ["table_name", "column_name"], "output_table_name": "z_score_table", "output_column_names": ["z_score_column"]}. The number of output columns should be the same as the number of input columns.'
    },
    't_test': {
        'function': t_test,
        'description': 'Perform a t-test between two columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"]], "output_table_name": "t_test_table", "output_column_names": ["t_test_statistic", "t_test_pvalue"]}. The number of output columns should be two: one for the t-test statistic and one for the p-value.'
    },
    'correlation': {
        'function': correlation,
        'description': 'Compute the correlation between two columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"]], "output_table_name": "correlation_table", "output_column_names": ["correlation_coefficient"]}. The number of output columns should be one: the correlation coefficient.'
    },
    'skewness': {
        'function': skewness_computation,
        'description': 'Compute the skewness of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "skewness_table", "output_column_names": ["skewness_column1", "skewness_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'quantile': {
        'function': quantile_computation,
        'description': 'Compute the specified quantile of the specified columns across tables in the database. JSON format: {"category_column": ["table_name", "category_column"], "quantile": value, "output_table_name": "quantile_table", "output_column_names": ["quantile_column"]}. The number of output columns should be the same as the number of input columns.'
    },
    'normalization': {
        'function': normalization,
                'description': 'Normalize the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "normalization_table", "output_column_names": ["normalization_column1", "normalization_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'log_transformation': {
        'function': log_transformation,
        'description': 'Apply log transformation to the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "log_transformation_table", "output_column_names": ["log_transformation_column1", "log_transformation_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'exp_transformation': {
        'function': exp_transformation,
        'description': 'Apply exponential transformation to the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "exp_transformation_table", "output_column_names": ["exp_transformation_column1", "exp_transformation_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'pct_change': {
        'function': pct_change_computation,
        'description': 'Compute the percentage change of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "pct_change_table", "output_column_names": ["pct_change_column1", "pct_change_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'covariance': {
        'function': covariance_computation,
        'description': 'Compute the covariance between two columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"]], "output_table_name": "covariance_table", "output_column_names": ["covariance_column"]}. The number of output columns should be one: the covariance value.'
    },
    'iqr': {
        'function': iqr_computation,
        'description': 'Compute the interquartile range (IQR) of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "iqr_table", "output_column_names": ["iqr_column1", "iqr_column2", ...]}. The number of output columns should be the same as the number of input columns.'
    },
    'categories_sum': {
        'function': categories_sum,
        'description': 'Compute the sum of one column grouped by another column in the same table. JSON format: {"group_by_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "output_table_name": "categories_sum_table", "output_column_names": ["categories_sum_column"]}. The number of output columns should be two: one for the group_by column and one for the summed target column.'
    },
    'categories_mean': {
        'function': categories_mean,
        'description': 'Compute the mean of one column grouped by another column in the same table. JSON format: {"group_by_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "output_table_name": "categories_mean_table", "output_column_names": ["categories_mean_column"]}. The number of output columns should be two: one for the group_by column and one for the mean target column.'
    },
    'categories_min': {
        'function': categories_min,
        'description': 'Compute the minimum of one column grouped by another column in the same table. JSON format: {"group_by_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "output_table_name": "categories_min_table", "output_column_names": ["categories_min_column"]}. The number of output columns should be two: one for the group_by column and one for the min target column.'
    },
    'categories_max': {
        'function': categories_max,
        'description': 'Compute the maximum of one column grouped by another column in the same table. JSON format: {"group_by_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "output_table_name": "categories_max_table", "output_column_names": ["categories_max_column"]}. The number of output columns should be two: one for the group_by column and one for the max target column.'
    },
    'categories_count': {
        'function': categories_count,
        'description': 'Compute the count of one column grouped by another column in the same table. JSON format: {"group_by_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "output_table_name": "categories_count_table", "output_column_names": ["categories_count_column"]}. The number of output columns should be two: one for the group_by column and one for the count target column.'
    },
    'numerical_sum': {
        'function': numerical_sum,
        'description': 'Compute the sum of one column based on numerical splits of another column in the same table. JSON format: {"category_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "splits": [split_value1, split_value2, ...], "output_table_name": "numerical_sum_table", "output_column_names": ["numerical_sum_column"]}. The number of output columns should match the number of splits plus the target column.'
    },
    'numerical_mean': {
        'function': numerical_mean,
        'description': 'Compute the mean of one column based on numerical splits of another column in the same table. JSON format: {"category_column": ["table_name", "category_column"], "target_column": ["table_name", "value_column"], "splits": [split_value1, split_value2, ...], "output_table_name": "numerical_mean_table", "output_column_names": ["numerical_mean_column"]}. The number of output columns should match the number of splits plus the target column.'
    },
    'count': {
        'function': count,
        'description': 'Count the number of samples of the specified columns across tables in the database. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "output_table_name": "count_table", "output_column_names": ["count_column"]}. The number of output columns should be the same as the number of input columns.'
    },
    'Group_By': {
            'function': group_by_columns,
            'description': 'Group the data in a table by the specified columns. JSON format: {"table": "table_name", "group_by_columns": ["column_name1", "column_name2", ...]， "output_table_name": "group_by_table", "output_column_names": ["group_by_column1", "group_by_column2", ...]}. The number of output columns should be the same as the number of input columns. The output will be a DataFrame with the specified grouping columns included.'
        },
    'join_tables': {
        'function': join_tables,
        'description': 'Join multiple tables in the database based on the specified columns. JSON format: {"columns": [["table_name1", "column_name1"], ["table_name2", "column_name2"], ...], "how": "join_type", "output_table_name": "join_table"}. The "columns" parameter should include at least two tables and their corresponding columns to join on. The "how" parameter specifies the type of join (default is "inner"). The output will be a DataFrame with all column names kept the same.'
    },
    'arrange': {
        'function': arrange,
        'description': 'Arrange the rows of a specified table in the database based on a given column. JSON format: {"columns": [["table_name", "column_name"]], "descending": boolean, "output_table_name": "arrange_table"}. The "columns" parameter should specify the table and column to sort by. The "descending" parameter determines the sort order (default is ascending). The output will be a sorted DataFrame with all column names kept the same.'
    },

    'mutate' :{
    'function': mutate,
    'description': 'Create new columns in a specified table by applying functions to existing columns. Only include the new column name and the old column name used in the lambda function in "new_columns" param. JSON format: {"table": "table_name", "new_columns": {"new_column_name": {"old_column_name": "lambda_function"}}, "output_table_name": "mutate_table"}. Make sure to write the lambda_function into a string. The output will be a DataFrame with all column names kept the same.'
    },

    'filter': {
      'function': filter,
      'description': 'Apply filter to a specific column in a table of the database with given condition. JSON format: {"table": "table_name", "column": "column_name", "condition": "condition"}. ex. {"table": "table_name", "column": "column_name", "condition": "< 5"}. ex2. {"table": "table_name", "column": "column_name", "condition": "== "A""}.'
    },
    'select_columns': {
      'function': select_columns,
      'description': 'Select specific columns from a table in the database. JSON format: {"table": "table_name1", "column": ["column1", "column2", ...]}.'
    },
    'Time Difference': {
    'function': calculate_time_difference,
    'description': 'Calculate the time difference between two columns of times. JSON format: {"time1": {"table": "table1", "column": "column1"}, "time2": {"table": "table2", "column": "column2"}}. The function returns a list of time differences in minutes.'
    }
}




# Testing

In [ ]:
# Define test cases
test_cases = [
    ("Sum Columns", sum_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'sum_absence_experience',
        'output_column_names': ['sum_Student_absence_days_per_year', 'sum_Experience_years']
    }),
    ("Mean Columns", mean_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'mean_absence_experience',
        'output_column_names': ['mean_Student_absence_days_per_year', 'mean_Experience_years']
    }),
    ("Max Columns", max_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'max_absence_experience',
        'output_column_names': ['max_Student_absence_days_per_year', 'max_Experience_years']
    }),
    ("Min Columns", min_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'min_absence_experience',
        'output_column_names': ['min_Student_absence_days_per_year', 'min_Experience_years']
    }),
    ("Standard Deviation Columns", std_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'std_absence_experience',
        'output_column_names': ['std_Student_absence_days_per_year', 'std_Experience_years']
    }),
    ("Variance Columns", var_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'var_absence_experience',
        'output_column_names': ['var_Student_absence_days_per_year', 'var_Experience_years']
    }),
    ("Median Columns", median_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'median_absence_experience',
        'output_column_names': ['median_Student_absence_days_per_year', 'median_Experience_years']
    }),
    ("Product Columns", product_columns, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'product_absence_experience',
        'output_column_names': ['product_Student_absence_days_per_year', 'product_Experience_years']
    }),
    ("Sum of Squares Columns", sum_of_squares, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'sum_of_squares_absence_experience',
        'output_column_names': ['sum_of_squares_Student_absence_days_per_year', 'sum_of_squares_Experience_years']
    }),
    ("Mode Computation", mode_computation, {
        'columns': [['Students_routes', 'Score']],
        'output_table_name': 'mode_score',
        'output_column_names': ['mode_Score']
    }),
    ("Range Computation", range_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'range_absence_experience',
        'output_column_names': ['range_Student_absence_days_per_year', 'range_Experience_years']
    }),
    ("Sum of Absolute Differences", sum_abs_diff_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'sum_abs_diff_absence_experience',
        'output_column_names': ['sum_abs_diff_Student_absence_days_per_year']
    }),
    ("Cumulative Sum", cumulative_sum_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year']],
        'output_table_name': 'cumulative_sum_absence',
        'output_column_names': ['cumulative_sum_Student_absence_days_per_year']
    }),
    ("Regression", regression, {
        'x': [['Students_routes', 'Student_absence_days_per_year']],
        'y': ['Students_buses', 'Experience_years'],
        'output_table_name': 'regression_absence_experience',
        'output_column_names': ['regression_experience']
    }),
    ("Z-Score", z_score, {
        'column': ['Students_routes', 'Student_absence_days_per_year'],
        'output_table_name': 'z_score_absence',
        'output_column_names': ['z_score_Student_absence_days_per_year']
    }),
    ("T-Test", t_test, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 't_test_absence_experience',
        'output_column_names': ['t_test_Student_absence_days_per_year', 't_test_Experience_years']
    }),
    ("Correlation", correlation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'correlation_absence_experience',
        'output_column_names': ['correlation_Student_absence_days_per_year']
    }),
    ("Skewness Computation", skewness_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'skewness_absence_experience',
        'output_column_names': ['skewness_Student_absence_days_per_year', 'skewness_Experience_years']
    }),
    ("Quantile Computation", quantile_computation, {
        'category_column': ['Students_routes', 'Student_absence_days_per_year'],
        'quantile': 0.5,
        'output_table_name': 'quantile_absence',
        'output_column_names': ['quantile_Student_absence_days_per_year']
    }),
    ("Normalization", normalization, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'normalization_absence_experience',
        'output_column_names': ['normalization_Student_absence_days_per_year', 'normalization_Experience_years']
    }),
    ("Log Transformation", log_transformation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'log_transformation_absence_experience',
        'output_column_names': ['log_transformation_Student_absence_days_per_year', 'log_transformation_Experience_years']
    }),
    ("Exponential Transformation", exp_transformation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'exp_transformation_absence_experience',
        'output_column_names': ['exp_transformation_Student_absence_days_per_year', 'exp_transformation_Experience_years']
    }),
    ("Percentage Change Computation", pct_change_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'pct_change_absence_experience',
        'output_column_names': ['pct_change_Student_absence_days_per_year', 'pct_change_Experience_years']
    }),
    ("Covariance Computation", covariance_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'covariance_absence_experience',
        'output_column_names': ['covariance']
    }),

    ("Interquartile Range Computation", iqr_computation, {
        'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']],
        'output_table_name': 'iqr_absence_experience',
        'output_column_names': ['iqr_Student_absence_days_per_year', 'iqr_Experience_years']
    }),
    ("Categories Sum", categories_sum, {
        'group_by_column': ['Students_routes', 'Grade'],
        'target_column': ['Students_routes', 'Student_absence_days_per_year'],
        'output_table_name': 'categories_sum_grade_absence',
        'output_column_names': ['categories_sum_Grade', 'categories_sum_Student_absence_days_per_year']
    }),
    ("Categories Mean", categories_mean, {
        'group_by_column': ['Students_routes', 'Grade'],
        'target_column': ['Students_routes', 'Student_absence_days_per_year'],
        'output_table_name': 'categories_mean_grade_absence',
        'output_column_names': ['categories_mean_Grade', 'categories_mean_Student_absence_days_per_year']
    }),
    ("Categories Min", categories_min, {
        'group_by_column': ['Students_routes', 'Grade'],
        'target_column': ['Students_routes', 'Student_absence_days_per_year'],
        'output_table_name': 'categories_min_grade_absence',
        'output_column_names': ['categories_min_Grade', 'categories_min_Student_absence_days_per_year']
    }),
    ("Categories Max", categories_max, {
        'group_by_column': ['Students_routes', 'Grade'],
        'target_column': ['Students_routes', 'Student_absence_days_per_year'],
        'output_table_name': 'categories_max_grade_absence',
        'output_column_names': ['categories_max_Grade', 'categories_max_Student_absence_days_per_year']
    }),
    ("Categories Count", categories_count, {
        'group_by_column': ['Students_routes', 'Grade'],
        'target_column': ['Students_routes', 'Student_ID'],
        'output_table_name': 'categories_count_grade',
        'output_column_names': ['categories_count_Grade', 'categories_count_Student_ID']
    }),
    ("Numerical Sum", numerical_sum, {
        'category_column': ['Students_routes', 'Student_absence_days_per_year'],
        'target_column': ['Students_routes', 'Grade'],
        'splits': [2, 4, 6],
        'output_table_name': 'numerical_sum_absence_grade',
        'output_column_names': ['numerical_sum_absence', 'numerical_sum_grade']
    }),
    ("Numerical Mean", numerical_mean, {
        'category_column': ['Students_routes', 'Student_absence_days_per_year'],
        'target_column': ['Students_routes', 'Grade'],
        'splits': [2, 4, 6],
        'output_table_name': 'numerical_mean_absence_grade',
        'output_column_names': ['numerical_mean_absence', 'numerical_mean_grade']
    }),
    ("Count", count, {
        'columns': [['Students_routes', 'Student_ID'], ['Students_buses', 'Bus_ID']],
        'output_table_name': 'count_student_bus',
        'output_column_names': ['count_Student_ID', 'count_Bus_ID']
    }),
    ("Group By", group_by_columns, {
        'table': 'Students_routes',
        'group_by_columns': ['Gender', 'Grade'],
        'output_table_name': 'grouped_students_routes',
        'output_column_names': ['Gender', 'Grade', 'Count']
    }),
    ("Filter", filter, {
        'table': 'Students_routes',
        'column': 'Grade',
        'condition': '== 9',
        'output_table_name': 'filtered_students_routes'
    }),
    ("Select Columns", select_columns, {
        'table': 'Students_routes',
        'column': ['Student_ID', 'Grade'],
        'output_table_name': 'selected_students_routes'
    }),
    ("Join Tables", join_tables, {
        'columns': [['Students_routes', 'Student_ID'], ['Students_buses', 'Student_ID']],
        'how': 'inner',
        'output_table_name': 'joined_students_routes_buses'
    }),
    ("Arrange", arrange, {
        'columns': [['Students_routes', 'Grade']],
        'descending': True,
        'output_table_name': 'arranged_students_routes'
    }),
    ("Mutate", mutate, {
        "table": "Students_routes",
        "new_columns": {"double_grade": "lambda df: df['Grade'] * 2"},
        "output_table_name": "mutated_students_routes"
    })
]



# Running the test cases
for test_name, func, params in test_cases:
      result = func(dataframes, params)
      print(f"{test_name} ({params}):\n{result.to_string(index=False)}\n")



Sum Columns ({'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']], 'output_table_name': 'sum_absence_experience', 'output_column_names': ['sum_Student_absence_days_per_year', 'sum_Experience_years']}):
 sum_Student_absence_days_per_year  sum_Experience_years
                                37                    81

Mean Columns ({'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']], 'output_table_name': 'mean_absence_experience', 'output_column_names': ['mean_Student_absence_days_per_year', 'mean_Experience_years']}):
 mean_Student_absence_days_per_year  mean_Experience_years
                                3.7                    8.1

Max Columns ({'columns': [['Students_routes', 'Student_absence_days_per_year'], ['Students_buses', 'Experience_years']], 'output_table_name': 'max_absence_experience', 'output_column_names': ['max_Student_absence_days_per_year', 'max_Experience_years

/usr/local/lib/python3.10/dist-packages/pandas/core/arraylike.py:396: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
<ipython-input-5-d22a157a23d1>:348: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bins')[target_col].sum().reset_index()
<ipython-input-5-d22a157a23d1>:367: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = data.groupby('bins')[target_col].mean().reset_index()


Numerical Sum ({'category_column': ['Students_routes', 'Student_absence_days_per_year'], 'target_column': ['Students_routes', 'Grade'], 'splits': [2, 4, 6], 'output_table_name': 'numerical_sum_absence_grade', 'output_column_names': ['numerical_sum_absence', 'numerical_sum_grade']}):
numerical_sum_absence  numerical_sum_grade
               (2, 4]                    6
               (4, 6]                   14

Numerical Mean ({'category_column': ['Students_routes', 'Student_absence_days_per_year'], 'target_column': ['Students_routes', 'Grade'], 'splits': [2, 4, 6], 'output_table_name': 'numerical_mean_absence_grade', 'output_column_names': ['numerical_mean_absence', 'numerical_mean_grade']}):
numerical_mean_absence  numerical_mean_grade
                (2, 4]                   6.0
                (4, 6]                   7.0

Count ({'columns': [['Students_routes', 'Student_ID'], ['Students_buses', 'Bus_ID']], 'output_table_name': 'count_student_bus', 'output_column_names': ['count_Stu

# Chatbots

schema for our testing database

In [ ]:
schema = """
Schema for School Transportation System:

1. Students_routes:
    - Student_ID: STRING, Unique
    - Student_first_name: STRING
    - Student_last_name: STRING
    - Email: STRING
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Grade: INTEGER, Range [1, 12]
    - Ethnicity: STRING, Values ['Asian', 'White', 'Black', 'Native']
    - Home_address: STRING
    - City: STRING
    - State: STRING
    - Postal_code: INTEGER, Range [10000, 99999]
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Parent_name: STRING
    - Siblings: INTEGER, Range [0, 5]
    - Principle_name: STRING
    - Student_absence_days_per_year: INTEGER, Range [1, 10]
    - Score: STRING, Values ['A', 'B', 'C']
    - Route_ID: STRING, Unique
    - Route_number: INTEGER, Range [100, 130]
    - Start_location: STRING
    - End_location: STRING
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Distance(miles): INTEGER, Range [5, 8]
    - #_of_stops: INTEGER, Range [10, 20]
    - #_of_intersections: INTEGER, Range [1, 20]

2. Students_buses:
    - Student_ID: STRING, Unique
    - Student_first_name: STRING
    - Student_last_name: STRING
    - Email: STRING
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Grade: INTEGER, Range [1, 12]
    - Ethnicity: STRING, Values ['Asian', 'White', 'Black', 'Native']
    - Home_address: STRING
    - City: STRING
    - State: STRING
    - Postal_code: INTEGER, Range [10000, 99999]
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Parent_name: STRING
    - Siblings: INTEGER, Range [0, 5]
    - Principle_name: STRING
    - Student_absence_days_per_year: INTEGER, Range [1, 10]
    - Score: STRING, Values ['A', 'B', 'C']
    - Bus_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_make: STRING
    - Bus_model: STRING
    - Maintenac_date: STRING, Pattern 'XX/XX/202X'
    - Bus_capacity: INTEGER, Range [48, 60]
    - License_plate: STRING, Unique, Pattern 'LXXXXX'
    - Driver_ID: STRING, Unique
    - First_name: STRING
    - Last_name: STRING
    - Email: STRING
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Age: INTEGER, Range [35, 65]
    - Experience_years: INTEGER, Range [1, 15]

3. Students_1:
    - Student_ID: STRING, Unique
    - Student_first_name: STRING
    - Student_last_name: STRING
    - Email: STRING
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Grade: INTEGER, Range [1, 12]
    - Ethnicity: STRING, Values ['Asian', 'White', 'Black', 'Native']
    - Home_address: STRING
    - City: STRING
    - State: STRING
    - Postal_code: INTEGER, Range [10000, 99999]
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Parent_name: STRING
    - Siblings: INTEGER, Range [0, 5]
    - Principle_name: STRING
    - Student_absence_days_per_year: INTEGER, Range [1, 10]
    - Score: STRING, Values ['A', 'B', 'C']

4. Schedules_2:
    - Schedule_ID: STRING, Unique
    - Route_ID: STRING, Unique
    - Driver_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_ID: STRING, Unique
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Start_location: STRING
    - End_location: STRING

5. Schedules_1:
    - Schedule_ID: STRING, Unique
    - Route_ID: STRING, Unique
    - Driver_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_ID: STRING, Unique
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Start_location: STRING
    - End_location: STRING

6. Routes_1:
    - Route_ID: STRING, Unique
    - Route_number: INTEGER, Range [100, 130]
    - Start_location: STRING
    - End_location: STRING
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Distance(miles): INTEGER, Range [5, 8]
    - #_of_stops: INTEGER, Range [10, 20]
    - #_of_intersections: INTEGER, Range [1, 20]

7. Bus_routes:
    - Route_ID: STRING, Unique
    - Route_number: INTEGER, Range [100, 130]
    - Start_location: STRING
    - End_location: STRING
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Distance(miles): INTEGER, Range [5, 8]
    - #_of_stops: INTEGER, Range [10, 20]
    - #_of_intersections: INTEGER, Range [1, 20]
    - Bus_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_make: STRING
    - Bus_model: STRING
    - Maintenance_date: STRING, Pattern 'XX/XX/202X'
    - Bus_capacity: INTEGER, Range [48, 60]
    - License_plate: STRING, Unique, Pattern 'LXXXXX'
    - Driver_ID: STRING, Unique
    - First_name: STRING
    - Last_name: STRING
    - Email: STRING
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Age: INTEGER, Range [35, 65]
    - Experience_years: INTEGER, Range [1, 15]
    - On_schedule_rate(%): INTEGER, Range [70, 100]
    - Driver_absence_days_per_year: INTEGER, Range [0, 10]

8. Bus_routes_2:
    - Route_ID: STRING, Unique
    - Route_number: INTEGER, Range [100, 130]
    - Start_location: STRING
    - End_location: STRING
    - Morning_arrive_time: STRING, Pattern 'HH:MM AM
    - Afternoon_arrive_time: STRING, Pattern 'HH:MM PM
    - Distance(Miles): INTEGER, Range [5, 8]
    - #_of_stops: INTEGER, Range [10, 20]
    - #_of_intersections: INTEGER, Range [1, 20]
    - Bus_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_make: STRING
    - Bus_model: STRING
    - Maintenac_date: STRING, Pattern 'XX/XX/202X'
    - Bus_capacity: INTEGER, Range [48, 60]
    - License_plate: STRING, Unique, Pattern 'LXXXXX'
    - Driver_ID: STRING, Unique
    - First_name: STRING
    - Last_name: STRING
    - Email: STRING
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Age: INTEGER, Range [35, 65]
    - Experience_years: INTEGER, Range [1, 15]
    - On_schedule_rate(%): INTEGER, Range [70, 100]
    - Driver_absence_days_per_year: INTEGER, Range [0, 10]

9. Bus_incident:
    - School_year: STRING, Pattern '20XX-20XX'
    - Bus_breakdown_ID: STRING, Unique
    - Run_type: STRING, Value ['Special Ed AM Run', 'Pre-K/El']
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_make: STRING
    - Bus_model: STRING
    - Maintenac_date: STRING, Pattern 'XX/XX/202X'
    - Bus_capacity: INTEGER, Range [48, 60]
    - License_plate: STRING, Unique, Pattern 'LXXXXX'
    - Driver_ID: STRING, Unique
    - Route_ID: STRING, Unique
    - Route_number: INTEGER, Range [100, 130]
    - Reason: STRING, Value ['Heavy Traffic', 'Won't Start', 'Flat Tire', 'Heavy Traffic', 'Other']
    - Schools_serviced: STRING
    - Bus_created_on: STRING, Pattern 'XX/XX/202X'
    - How_long_delayed: STRING, Pattern 'XX MINS'
    - Number_of_students_on_the_bus: INTEGER, Range [0, 15]
    - Has_contractor_notified_schools: STRING, Value ['Yes', 'No']
    - Has_contractor_notified_parents: STRING, Value ['Yes', 'No']
    - Has_driver_alerted_OPT: STRING, Value ['Yes', 'No']
    - Incident_occurred_On: STRING, Pattern 'XX/XX/202X'
    - Informed_On: STRING, Pattern 'XX/XX/202X'
    - On_schedule_rate(%): INTEGER, Range [50, 100]
    - Driver_absence_days_per_year: INTEGER, Range [0, 10]

10. Bus_data_1:
    - Bus_ID: STRING, Unique
    - Bus_number: INTEGER, Unique, Range [100, 199]
    - Bus_make: STRING
    - Bus_model: STRING
    - Maintenac_date: STRING, Pattern 'XX/XX/202X'
    - Bus_capacity: INTEGER, Range [48, 60]
    - License_plate: STRING, Unique, Pattern 'LXXXXX'
    - Fuel_type: STRING, Value ['Diesel', 'Eletric']
    - Driver_ID: STRING, Unique
    - First_name: STRING
    - Last_name: STRING
    - Email: STRING
    - Phone_number: STRING, Unique, Pattern 'XXX-XXX-XXXX'
    - Gender: STRING, Values ['Male', 'Female', 'Genderqueer', 'Non-binary', 'Agender', 'Genderfluid', 'Bigender', 'Polygender']
    - Age: INTEGER, Range [35, 65]
    - Experience_years: INTEGER, Range [1, 15]
    - On_schedule_rate(%): INTEGER, Range [70, 100]
    - Driver_absence_days_per_year: INTEGER, Range [0, 10]

Relationships:
- A Bus can be assigned to multiple Routes (many-to-many through BusAssignments).
- A Route can have multiple Buses assigned to it (many-to-many through BusAssignments).
- A Driver can drive multiple Routes (one-to-many through Schedules).
- A Route can be driven by multiple Drivers (one-to-many through Schedules).
- A Schedule links a Route, a Driver, and a Bus (many-to-many through Schedules).
"""

# User版本

In [ ]:

class Main_bot:
    def __init__(self, testing, model=MODEL, api_key=openai.api_key, computations_supported=computations_supported, schema=schema, dataframes=dataframes, user_id=None, folder_path=None):
        self.client = openai.OpenAI(api_key=api_key)
        self.model = model
        self.testing = testing
        self.computations_supported = computations_supported
        self.dataframes = dataframes
        self.schema = schema
        self.user_id = user_id
        self.folder_path = folder_path

        self.functions_called = []
        self.function_used_count = {}
        self.function_sequences = []

        self.conversation_history = []
        self.complete_conversation_history = []
        self.initial_system_description = '''You are a chatbot system designed to take user input and allow the assistant to chat with the user.
        Meanwhile, the system constantly generates suggestions on what to do next before the assistant produces any output.
        After receiving user input, the system will generate a list of the most likely functions needed to continue the conversation.
        For example, it might suggest functions such as [{"function_name": "function1", "params": {"columns": [["table2", "col1"], ["table5", "col3"]], "output_table_name": "new_table", "output_column_names": ["col1_new", "col2_new"]}, "probability": "90"}, ...].
        Ensure that the column input exists in the specified table. Pay close attention to the schema to avoid any mistakes.
        The output will be a JSON-formatted list of dictionaries, each containing a function name, its parameters, a probability score, an output table name, and output column names.
        The computations will be performed as instructed, and the results will be provided to the system through the system role.
        The assistant will then use this information to generate a direct response to the user's request, focusing only on the insightful results.
        If a computation involves multiple steps, each step will output a new table, and subsequent steps can use these tables as part of the dataframes or database.
        For example, converting numerical age to categorical and then performing classification on the new categorical age column, with filters applied to select specific categories before classification.'''

        self.system_instruction_reminder = '''\nTask description:
        Generate a dict-like output mapping function names and their parameters to probabilities as described before.
        Include an "output_table_name" key and an "output_column_names" key within the params.
        Decide the probability based on previous conversations and the most recent user's request.
        No matter what the user input is, your job is always the same; you are not supposed to respond freely.
        Ensure the dict-like output is the only content so it can be easily jsonified. The output format is always:
        [{"function_name": "function1", "params": {"columns": [["table2", "col1"], ["table5", "col3"]], "output_table_name": "new_table", "output_column_names": ["col1_new", "col2_new"]}, "probability": "90"}, ...].
        For example: [{"function_name": "count", "params": {"columns": [["Drivers", "Driver_ID"]], "output_table_name": "count_table", "output_column_names": ["Driver_ID_count"]}, "probability": "95"}].
        Remember to enclose everything in a pair of [ ]. When there are no valuable functions to be called, return an empty list.
        Ensure the column input exists in the specified table. Pay close attention to the schema to avoid any mistakes.
        Include "filters" when necessary to refine the data before performing computations. Filters should be in the format: "filters": [{"table": "table1", "col": "col1", "range": ["category1", "category3"], "exclude": true}, {"table": "table2", "col": "col7", "range": [[0, 9], [13, 90]], "exclude": false}]. In each range list, one value can be INF or NINF indicating infinity and negative infinity.
        The order of the column names in "output_column_names" should match the order of the columns processed as specified in the "columns" input.
        Make sure that the inputs comes from the schema I gave you or output tables in previous function'''

        self.chat_assistant_reminder = '''\nTask description:
        Based on the user's last request and the computation results provided to you, come up with an insightful report answering the user's question or continue the conversation if there was no obvious question.
        If no computation result is provided, then just chat with the user as you like.
        When multiple results are provided, identify the most useful information and present it in your own way.
        Whenever you feel it's necessary to include a graph in your answer, use the format ||chartjs: ...||, where ... contains the Chart.js configuration in JSON format.
        You only need to specify the data and type in the Chart.js configuration; additional parameters like background color are not necessary.
        In your text, never explicitly cite or mention the graph you might use because if the graphing steps on the backend fail, your output will remain coherent.
        Be sure to use graph more often when talking about data.
        '''

        self.intial_assistant_request = "Hello, I am here to help you to learn about your database, provide you with interesting and helpful insights from your data"

        self.first_run = True
        self.log_conversation('system', "\nThis is a description of the database I have:\n" + schema)
        self.log_conversation(
            'system',
            "\nHere's all the possible computations or functions that you might need to use:\n" +
            "\n".join([f"{key}: {value['description']}" for key, value in self.computations_supported.items()]))
        self.log_conversation('system', self.initial_system_description)
        self.assistant_output = self.intial_assistant_request

    def log_conversation(self, role, content):  # role range assistant system user
        self.conversation_history.append({'role': role, 'content': content})
        self.complete_conversation_history.append({'role': role, 'content': content})

    def generate(self, use_complete_log=False):
        if use_complete_log:
            response = self.client.chat.completions.create(
              messages=self.complete_conversation_history,
              model=self.model,
              temperature=0
          )

        # Call the AI model to suggest visualization based on the data_json
        else:
            response = self.client.chat.completions.create(
              messages=self.conversation_history,
              model=self.model,
              temperature=0
          )

        # Get the model's response and log it
        visualization_response = response.choices[0].message.content.strip()
        return visualization_response

    def filter_and_rank_functions(self, response):
        # Strip everything before the first `[` and after the last `]`
        start = response.find('[')
        end = response.rfind(']') + 1
        if start == -1 or end == -1:
            raise ValueError("No valid JSON array found in response.")

        json_text = response[start:end]
        function_probs = json.loads(json_text)
        return function_probs

    def get_computation_results(self, computation_to_use_rank_dict_txt_response):
        # Get the ranked functions to use
        computation_to_use_rank_dict = self.filter_and_rank_functions(computation_to_use_rank_dict_txt_response)
        tem_dataframes = self.dataframes.copy()
        results = []

        for item in computation_to_use_rank_dict:
            computation_name = item["function_name"]
            params = item["params"]
            output_table_name = params.get("output_table_name", None)

            if computation_name in self.computations_supported:
                function = self.computations_supported[computation_name]['function']
                self.function_used_count[computation_name] = self.function_used_count.get(computation_name, 0) + 1

                if self.testing:
                    print("Currently processing:" + computation_name)
                    # Perform the computation
                    result = function(tem_dataframes, params)
                    print(result)
                    if output_table_name:
                        tem_dataframes[output_table_name] = result
                        results.append(f"{computation_name} ({params}):\n{result.to_string(index=False)}\n")
                    else:
                        results.append(f"{computation_name} ({params}):\n{result.to_string(index=False)}\n")
                else:
                    try:
                        # Perform the computation
                        result = function(tem_dataframes, params)
                        self.functions_called.append({
                            'function': computation_name,
                            'error': False,
                            'inputs': params
                        })
                        if output_table_name:
                            tem_dataframes[output_table_name] = result
                            results.append(f"{computation_name} ({params}):\n{result.to_string(index=False)}\n")
                        else:
                            results.append(f"{computation_name} ({params}):\n{result.to_string(index=False)}\n")
                    except Exception as e:
                        # Log the error details
                        self.functions_called.append({
                            'function': computation_name,
                            'error': True,
                            'inputs': params
                        })
                        # Do not append error details to results in non-testing mode
                        pass
        # Join all results into a single string
        computation_results_str = "\n".join(results)

        if not results:
            return "All given functions are facing errors, please respond freely as you see fit."

        return computation_results_str

    def generate_and_display_chart(self, config, width=500, height=300, version='2.9.4'):
        qc = QuickChart()
        qc.width = width
        qc.height = height
        qc.version = version
        qc.config = config

        # Fetch the image as bytes directly from QuickChart
        image_data = qc.get_bytes()

        # Display the image in the notebook
        display(Image(image_data))

    def process_output(self):
        input_string = self.assistant_output
        # Split the input string by the graph indicator
        chunks = input_string.split('||')

        for chunk in chunks:
            if chunk.startswith('chartjs:'):
                # Extract the JSON part of the chunk
                json_part = chunk[len('chartjs:'):]

                try:
                    # Convert the JSON part to a dictionary
                    config = json.loads(json_part)
                    # Generate and display the chart
                    self.generate_and_display_chart(config)
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
            else:
                # Print the text chunk
                print(chunk)



    def generate_function_sequences(self):
        sequences = []
        current_sequence = []
        error_occur = None

        for func in self.functions_called:
            current_sequence.append(func['function'])
            if func['error'] and not error_occur:
                error_occur = func['function']

        sequences.append({
            'sequence': ' -> '.join(current_sequence),
            'error_occur': error_occur if error_occur else 'None'
        })

        self.function_sequences = sequences

    def execute(self):
        while True:
            # get user request and log it
            self.log_conversation('assistant', self.assistant_output)
            self.process_output()
            print('\n')
            user_request = input('\n')
            self.log_conversation('user', user_request)

            # system tell which function to use, don't log this step
            self.log_conversation('system', self.system_instruction_reminder)
            computation_to_use_rank_dict_txt_response = self.generate()  # text json-like

            if self.testing:
                print("computation_to_use_rank_dict_txt_response:", computation_to_use_rank_dict_txt_response)

            computation_results = self.get_computation_results(computation_to_use_rank_dict_txt_response)

            # system provide computation results, and remind assistant's task, to generate report
            self.log_conversation('system', computation_results)
            self.log_conversation('system', self.chat_assistant_reminder)

            # assistant output report using given data
            self.assistant_output = self.generate()

            self.conversation_history.pop()
            self.conversation_history.pop()
            print("\n\n\n\n\n")
    def record(self):
        # Ensure the directory exists
        logs_directory = os.path.join(self.folder_path, self.user_id, 'logs')
        if not os.path.exists(logs_directory):
            os.makedirs(logs_directory)

        # Get the current timestamp for unique filenames
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Save complete conversation history
        conversation_file = os.path.join(logs_directory, f'log_{timestamp}.json')
        with open(conversation_file, 'w') as file:
            json.dump(self.complete_conversation_history, file, indent=4)

        # Save function sequences
        function_sequences_file = os.path.join(logs_directory, f'functions_{timestamp}.json')
        with open(function_sequences_file, 'w') as file:
            json.dump(self.function_sequences, file, indent=4)

        # File paths for user-specific CSVs
        user_folder_path = os.path.join(self.folder_path, self.user_id)
        functions_called_file_user = os.path.join(user_folder_path, 'functions_called.csv')
        functions_count_file_user = os.path.join(user_folder_path, 'functions_count.csv')
        function_sequences_file_user = os.path.join(user_folder_path, 'function_sequences.csv')

        # File paths for global CSVs
        functions_called_file_global = os.path.join(self.folder_path, 'functions_called.csv')
        functions_count_file_global = os.path.join(self.folder_path, 'functions_count.csv')
        function_sequences_file_global = os.path.join(self.folder_path, 'function_sequences.csv')

        # Helper function to load existing data
        def load_existing_data(file_path, fieldnames):
            data = []
            if os.path.exists(file_path):
                with open(file_path, mode='r', newline='') as file:
                    reader = csv.DictReader(file, fieldnames=fieldnames)
                    next(reader)  # Skip header
                    data = list(reader)
            return data

        # Helper function to save data
        def save_data(file_path, data, fieldnames):
            with open(file_path, mode='w', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=fieldnames)
                writer.writeheader()
                for item in data:
                    writer.writerow(item)

        # Load existing function counts (user-specific and global)
        function_counts_user = load_existing_data(functions_count_file_user, ['function', 'count'])
        function_counts_global = load_existing_data(functions_count_file_global, ['function', 'count'])

        # Save or update function counts (user-specific and global)
        for function_name, count in self.function_used_count.items():
            found_user = False
            for item in function_counts_user:
                if item['function'] == function_name:
                    item['count'] = int(item['count']) + count
                    found_user = True
                    break
            if not found_user:
                function_counts_user.append({'function': function_name, 'count': count})

            found_global = False
            for item in function_counts_global:
                if item['function'] == function_name:
                    item['count'] = int(item['count']) + count
                    found_global = True
                    break
            if not found_global:
                function_counts_global.append({'function': function_name, 'count': count})

        save_data(functions_count_file_user, function_counts_user, ['function', 'count'])
        save_data(functions_count_file_global, function_counts_global, ['function', 'count'])

        # Load existing functions_called (user-specific and global)
        functions_called_user = load_existing_data(functions_called_file_user, ['function', 'error', 'inputs', 'count'])
        functions_called_global = load_existing_data(functions_called_file_global, ['function', 'error', 'inputs', 'count'])

        # Save or update functions_called (user-specific and global)
        def update_functions_called(existing_functions, new_functions):
            for func in new_functions:
                found = False
                for existing_func in existing_functions:
                    if existing_func['function'] == func['function'] and existing_func['inputs'] == func['inputs']:
                        existing_func['error'] = func['error']
                        existing_func['count'] = int(existing_func['count']) + 1
                        found = True
                        break
                if not found:
                    func['count'] = 1
                    existing_functions.append(func)
            return existing_functions

        functions_called_user = update_functions_called(functions_called_user, self.functions_called)
        functions_called_global = update_functions_called(functions_called_global, self.functions_called)

        save_data(functions_called_file_user, functions_called_user, ['function', 'error', 'inputs', 'count'])
        save_data(functions_called_file_global, functions_called_global, ['function', 'error', 'inputs', 'count'])

        # Load existing function sequences (user-specific and global)
        function_sequences_user = load_existing_data(function_sequences_file_user, ['sequence', 'error_occur', 'count'])
        function_sequences_global = load_existing_data(function_sequences_file_global, ['sequence', 'error_occur', 'count'])

        # Generate and save function sequences (user-specific and global)
        self.generate_function_sequences()

        def update_function_sequences(existing_sequences, new_sequences):
            for seq in new_sequences:
                found = False
                for existing_seq in existing_sequences:
                    if existing_seq['sequence'] == seq['sequence']:
                        existing_seq['error_occur'] = seq['error_occur']
                        existing_seq['count'] = int(existing_seq['count']) + 1
                        found = True
                        break
                if not found:
                    seq['count'] = 1
                    existing_sequences.append(seq)
            return existing_sequences

        function_sequences_user = update_function_sequences(function_sequences_user, self.function_sequences)
        function_sequences_global = update_function_sequences(function_sequences_global, self.function_sequences)

        save_data(function_sequences_file_user, function_sequences_user, ['sequence', 'error_occur', 'count'])
        save_data(function_sequences_file_global, function_sequences_global, ['sequence', 'error_occur', 'count'])

    def generate_summary_report(self):
        # Generate a summary report from the complete conversation history
        summary_prompt = '''Generate a summarizing report of the conversation history. Include insights and graphs where relevant using the ||chartjs: ...|| format for graphs.'''
        self.log_conversation('system', summary_prompt)

        # Generate the summary using the complete conversation history
        summary_response = self.generate()

        # Split the summary response by the graph indicator
        summary_chunks = summary_response.split('||')

        # Display the summary with graphs
        # self.display_summary_with_graphs(summary_chunks)

        # Ensure the logs directory exists
        logs_directory = os.path.join(self.folder_path, self.user_id, 'logs')
        if not os.path.exists(logs_directory):
            os.makedirs(logs_directory)

        # Save the summary response as a text file
        summary_file = os.path.join(logs_directory, f'summary_{datetime.now().strftime("%Y%m%d_%H%M%S")}.txt')
        with open(summary_file, 'w') as file:
            file.write(summary_response)

    def display_summary_with_graphs(self, summary_chunks):
        for chunk in summary_chunks:
            if chunk.startswith('chartjs:'):
                json_part = chunk[len('chartjs:'):]
                try:
                    config = json.loads(json_part)
                    qc = QuickChart()
                    qc.config = config
                    image_url = qc.get_short_url()
                    display(Image(image_url))
                except json.JSONDecodeError as e:
                    print(f"Error decoding JSON: {e}")
            else:
                print(chunk)

    def check_and_update_required_functions(self, show=False):
        # Ensure the directory exists
        outermost_directory = self.folder_path
        required_functions_file = os.path.join(outermost_directory, 'required_functions.csv')

        # Read existing functions if the file exists
        existing_functions = []
        if os.path.exists(required_functions_file):
            with open(required_functions_file, mode='r', newline='') as file:
                reader = csv.DictReader(file)
                existing_functions = list(reader)

        # Ask the system to identify required functions based on the complete log history
        required_functions_prompt = '''Based on the complete conversation history, identify any specific user requests that are not supported by the current set of functions or their combinations. Only include the functions that are not already supported. Provide the names and descriptions of the required functions in the following format: [{"function_name": "example_function_name", "description": "This is an example description of the function."}, ...]. If a required function is very similar to any existing function descriptions, copy the existing function name and description.'''
        self.log_conversation('system', required_functions_prompt)

        required_functions_response = self.generate()
        if show:
            print(required_functions_response)
        try:
            new_required_functions = json.loads(required_functions_response)
        except json.JSONDecodeError:
            print("Error decoding JSON response for required functions.")
            return

        # Update existing functions with new required functions
        for new_function in new_required_functions:
            found = False
            for existing_function in existing_functions:
                if (existing_function['function_name'] == new_function['function_name'] and
                        existing_function['description'] == new_function['description']):
                    existing_function['count'] = int(existing_function['count']) + 1
                    found = True
                    break
            if not found:
                new_function['count'] = 1
                existing_functions.append(new_function)

        # Save updated required functions back to the file
        with open(required_functions_file, mode='w', newline='') as file:
            fieldnames = ['function_name', 'description', 'count']
            writer = csv.DictWriter(file, fieldnames=fieldnames)
            writer.writeheader()
            for func in existing_functions:
                writer.writerow(func)


    def print_conversation(self):
        for entry in self.conversation_history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")

    def print_complete_conversation(self):
        for entry in self.complete_conversation_history:
            print(f"{entry['role'].capitalize()}: {entry['content']}")

chat = Main_bot(testing=False, user_id='user123', folder_path=base_path+'saved_functions_log/')
chat.execute()


Hello, I am here to help you to learn about your database, provide you with interesting and helpful insights from your data



scscasca


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: xxx # us****************** key. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [ ]:

# Call the record method to save the logs and function sequences
chat.record()

# Generate and print the summary report with graphs
chat.generate_summary_report()

# Check and update required functions based on the complete log history
chat.check_and_update_required_functions(True)
